In [1]:
import tskit, msprime, pyslim
from IPython.display import SVG, display
import numpy as np
import os, subprocess, statistics
#import matplotlib.pyplot as plt

In [5]:
print("BurnInType","ScalingFactor","GenomeSize","RepNum","AvgMissCoalEvents","PropMissBranchLen","AvgMissCoalEvents_4N","PropMissBranchLen_4N", sep='\t')
for t in [5, 10, 20]:
    for c in [10, 5, 1]:
        N=7310/c
        rr=1e-8*c
        for G in ["1e5", "1e6", "1e7"]:
            for n in range(1,3):

                #ts = tskit.load(f'C:\\Users\\tferrari\\Desktop\\SlimBenchmark\\human\\out\\burn-ins\\humanBench_burnin{n}_{t}Ne_c{c}_{G}.trees')
                ts = tskit.load(f'C:\\Users\\tferrari\\Desktop\\SlimBenchmark\\human\\out\\burnins_wselection\\humanBench_burnin{n}_{t}Ne_c{c}_{G}_wsel.trees')
                
                # Measure ts stats per position
                events_for_pos = np.zeros(int(ts.sequence_length)) 
                length_for_pos = np.zeros(int(ts.sequence_length))
                for tree in ts.trees():
                    left, right = map(int, tree.interval)
                    events_for_pos[left: right] = tree.num_roots-1 # Number of missing coalescence events
                    length_for_pos[left: right] = tree.total_branch_length # Tree length
                    
                # Decapitate tree at time = 4N
                dts = ts.decapitate(time = (4*N))
                
                # Measure decapitated ts stats per position
                dts_events_for_pos = np.zeros(int(dts.sequence_length))
                dts_length_for_pos = np.zeros(int(dts.sequence_length))
                for tree in dts.trees():
                    left, right = map(int, tree.interval)
                    dts_events_for_pos[left: right] = tree.num_roots-1
                    dts_length_for_pos[left: right] = tree.total_branch_length
                
                # Recapitate tree
                rts = pyslim.recapitate(ts,
                                        recombination_rate=rr,
                                        ancestral_Ne=N)
                assert(max([t.num_roots for t in rts.trees()]) == 1)
                
                # Measure fully coalesced tree stats per position
                rts_length_for_pos = np.zeros(int(rts.sequence_length))
                for tree in rts.trees():
                    length = tree.total_branch_length
                    left, right = map(int, tree.interval)
                    rts_length_for_pos[left: right] = length
                
                # Summarize stats over all positions
                avg_events = statistics.mean(events_for_pos) # Num missing coalescence events averaged across positions
                prop_length = 1 - sum(length_for_pos)/sum(rts_length_for_pos) # Proportion of missing ts branch length

                # Summarize stats at Time=4N
                T4N_avg_events = statistics.mean(dts_events_for_pos) # Num missing coalescence events averaged across positions
                T4N_prop_length = 1 - sum(dts_length_for_pos)/sum(rts_length_for_pos) # Proportion of missing ts branch length
                
                # Print
                print(t,c,G,n,avg_events,prop_length,T4N_avg_events,T4N_prop_length,sep='\t')

BurnInType	ScalingFactor	GenomeSize	RepNum	AvgMissCoalEvents	PropMissBranchLen	AvgMissCoalEvents_4N	PropMissBranchLen_4N
5	10	1e5	1	0.15776	0.039138641997551904	0.17699	0.07628377980418177
5	10	1e5	2	0.02012	0.004086243466259187	0.02152	0.037491016416316025
5	10	1e6	1	0.171219	0.01605730252722737	0.309905	0.05591587958005351
5	10	1e6	2	0.085825	0.012891774348716023	0.180233	0.04987912355495494
5	10	1e7	1	0.0922908	0.01198854954507178	0.1724519	0.05272399116688964
5	10	1e7	2	0.0971772	0.01344047395689918	0.1916922	0.05416828366551296
5	5	1e5	1	0.22568	0.011206306249276321	0.29184	0.0486851993559716
5	5	1e5	2	0.08797	0.013504215875171921	0.08797	0.04730688894290114
5	5	1e6	1	0.179525	0.026257168607289105	0.316912	0.06291086725892725
5	5	1e6	2	0.19251	0.023963103263531127	0.277168	0.05951973075503558
5	5	1e7	1	0.1014841	0.011365997883836965	0.1939449	0.04720067848896503
5	5	1e7	2	0.1041077	0.01489127726783157	0.2096105	0.05141456172854775
5	1	1e5	1	0.17259	0.006999091552258863	0.71086	0.0